In [1]:
!pip install python-terrier

     |████████████████████████████████| 95 kB 2.5 MB/s 
     |████████████████████████████████| 1.1 MB 39.4 MB/s 
     |████████████████████████████████| 69 kB 6.8 MB/s 
     |████████████████████████████████| 255 kB 47.1 MB/s 
     |████████████████████████████████| 1.8 MB 43.5 MB/s 
     |████████████████████████████████| 596 kB 36.0 MB/s 
     |████████████████████████████████| 72 kB 1.1 MB/s 
     |████████████████████████████████| 6.3 MB 35.2 MB/s 
     |████████████████████████████████| 294 kB 50.5 MB/s 
     |████████████████████████████████| 126 kB 47.9 MB/s 
     |████████████████████████████████| 291 kB 50.4 MB/s 
     |████████████████████████████████| 45 kB 3.0 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.7.1-py3-none-any.whl size=102452 sha256=183805a12129f515c995289fb9a84571ef8454f8d42eec190043b9e54dc1029f
  Stored in directory: /root/.cache/pip/wheels/c0/9a/c8/1c2d9ec6a1494bb54f47e0d2627b5ed7b2de704b66723d3417
  Created wheel for ir-measures: filen

In [2]:
import pyterrier as pt
import pandas as pd
import os
from pyterrier.measures import *
import re

In [3]:
foodresults = pd.read_csv("foodresults.csv", encoding="latin-1")
index_result = pd.read_csv("index_result.csv")
df = pd.read_csv("dataset_with_relevance.csv",encoding="latin-1")

In [4]:
if not pt.started():
  pt.init()

terrier-assemblies 5.6 jar-with-dependencies not found, downloading to /root/.pyterrier...
Done
terrier-python-helper 0.0.6 jar not found, downloading to /root/.pyterrier...
Done
PyTerrier 0.7.1 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)


In [5]:
import re
s="sdfgfs$%$#%#$9998 sdfg"
word1 = " ".join(re.findall("[a-zA-Z]+", s))
word1

'sdfgfs sdfg'

In [6]:
data=[]
for index,row in df.iterrows():
  query = " ".join(re.findall("[a-zA-Z]+", row["query"]))
  recipe_id = row["recipe_id"]
  recipe_name = " ".join(re.findall("[a-zA-Z]+", row["recipe_name"]))
  Relevance = row["Relevance"]
  ingredients = " ".join(re.findall("[a-zA-Z]+", str(row["ingredients"])))
  recipe = " ".join(re.findall("[a-zA-Z]+", row["recipe"]))
  rate = row["rate"]
  views = row["views"]
  description = " ".join(re.findall("[a-zA-Z]+", row["description"]))
  keywords = " ".join(re.findall("[a-zA-Z]+", str(row["keywords"])))
  calories = row["calories"]
  fatContent = row["fatContent"]
  proteinContent = row["proteinContent"]
  data.append([query,recipe_id,recipe_name,Relevance,ingredients,recipe,
               rate,views,description,keywords,calories,fatContent,proteinContent])
new_df = pd.DataFrame(data,columns=["query","docno","recipe_name","Relevance","ingredients","recipe",
               "rate","views","description","keywords","calories","fatContent","proteinContent"])

In [7]:
new_df.head(6)

,query,docno,recipe_name,Relevance,ingredients,recipe,rate,views,description,keywords,calories,fatContent,proteinContent
0,tomato,6492,Macaroni and Tomatoes,5,large potatoes teaspoon salt eggs lightly beat...,Scrub potatoes well but do not peel In a sauce...,4.5,6,Make and share this Potato Plum Dumplings reci...,Vegetable Hungarian European Low Protein Healt...,270.6,8.9,5.8
1,tomato,50060,Tofu Stuffed Cherry Tomatoes,5,onion diced garlic cloves minced carrot diced ...,Sweat onion and garlic in small amount of oil ...,0.0,0,Creamy fat free low cal and a serving of veggi...,Lactose Free Low Protein Low Cholesterol Egg F...,107.6,0.8,3.8
2,tomato,54375,Canned Tomatoes Vine Ripened Tomatoes in Tomat...,5,butter recipe cake mix baked in a greased x cm...,For best results bake the cake the day before ...,0.0,0,Make and share this Lamingtons recipe from Foo...,Low Protein Low Cholesterol Healthy Kid Friend...,179.2,3.1,1.3
3,tomato,11958,Chilli Tomato Sauce Tomato Salsa,5,large mushrooms cup breadcrumbs cup parmesan c...,clean mushrooms with damp cloth remove stems a...,5.0,1,Make and share this Moms Stuffed Mushrooms rec...,Healthy Mins For Large Groups,16.8,0.7,1.4
4,tomato,51985,Cherry Tomato Planets,5,teaspoons sesame oil tablespoons gingerroot sh...,Place the sesame oil in a hot wok with the gin...,0.0,0,This is from the Donna Hay cookbook quot Cooki...,Asian Low Protein Low Cholesterol Healthy Mins...,86.8,2.3,0.5
5,tomato,16912,BLANCHING TOMATOES for COOKING amp CANNING,4,ounces large shrimp peeled and deveined tables...,Heat the oil in a large heavy bottomed skillet...,0.0,0,Make and share this Cilantro Lime Shrimp recip...,Very Low Carbs Healthy Mins,165.6,5.4,24.6


In [9]:

data=[]
foodresults["docno"] = range(0,len(foodresults))
foodresults["recipe_id"] = range(0,len(foodresults))
foodresults.head(5)
for index,row in foodresults.iterrows():
  docno = "d"+str(row["docno"])
  recipe_id = str(row["recipe_id"])
  recipe_name = " ".join(re.findall("[a-zA-Z]+", row["name"]))
  ingredients = " ".join(re.findall("[a-zA-Z]+", str(row["ingredients"])))
  recipe = " ".join(re.findall("[a-zA-Z]+", str(row["recipe"])))
  rate = row["rate"]
  views = row["views"]
  description = " ".join(re.findall("[a-zA-Z]+", row["description"]))
  keywords = " ".join(re.findall("[a-zA-Z]+", str(row["keywords"])))
  calories = row["calories"]
  fatContent = row["fatContent"]
  proteinContent = row["proteinContent"]
  data.append([recipe_id,recipe_name,ingredients,recipe,
               rate,views,description,keywords,calories,fatContent,proteinContent])
whole_df = pd.DataFrame(data,columns=["docno","recipe_name","ingredients","recipe",
               "rate","views","description","keywords","calories","fatContent","proteinContent"])

In [10]:
whole_df.head(5)

,docno,recipe_name,ingredients,recipe,rate,views,description,keywords,calories,fatContent,proteinContent
0,0,Chinese Fried Rice,cup finely chopped onion tablespoons oil egg l...,Heat tbsp oil in wok add chopped onions and st...,5.0,541,This Chinese fried rice has the flavor those o...,Pork Poultry Rice Meat Chinese Asian Low Chole...,497.8,16.3,22.3
1,1,Perfect Pork Tenderloin,lb boneless pork tenderloin salt and pepper dr...,NOTE nbsp The success of this cooking method w...,5.0,434,My way to cook a pork tenderloin or a boneless...,Meat Very Low Carbs High Protein Healthy High ...,136.2,4.0,23.4
2,2,Pulled Pork Crock Pot,lbs pork roast shoulder or butt large onions c...,Slice one onion and place in crock pot Put in ...,5.0,642,I found this pulled pork recipe years ago and ...,Meat Healthy Beginner Cook Easy,335.1,6.9,40.2
3,3,The Most Wonderful Gingerbread Cookies,cups all purpose flour teaspoons baking powder...,In a small bowl whisk together flour baking po...,4.5,559,This is my very favorite gingerbread cookie re...,Cookie Brownie Healthy Christmas Oven Hours,134.2,3.2,1.9
4,4,Restaurant Teriyaki Sauce,cup soy sauce cup water teaspoon ground ginger...,Mix all but cornstarch and c water in a sauce ...,5.0,341,You know the nice sweet thick Teriyaki sauce y...,Low Protein Low Cholesterol Healthy Mins Easy,290.5,0.1,5.3


In [12]:
whole_index_dir = "./whole_docs_index"
whole_indexer = pt.DFIndexer(whole_index_dir, overwrite=True)
whole_index_ref = whole_indexer.index(whole_df["recipe"], whole_df["docno"],
                    whole_df["recipe_name"], whole_df["keywords"], 
                    whole_df["description"], whole_df["ingredients"])
whole_index_ref.toString()

'./whole_docs_index/data.properties'

In [13]:
whole_index = pt.IndexFactory.of(whole_index_ref)

In [14]:
topics = pd.read_csv("topics.csv")
qrels = pd.read_csv("qrel.csv")
qrels = qrels.drop(columns=["iteration"])

In [15]:
topics.head(5)

,qid,query
0,1,apple
1,2,bacon
2,3,baking soda
3,4,barbecue ribs
4,5,beef


In [16]:
topics["qid"] = topics["qid"].astype(str)

In [17]:
topics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   qid     30 non-null     object
 1   query   30 non-null     object
dtypes: object(2)
memory usage: 608.0+ bytes


In [18]:
qrels["qid"] = qrels["qid"].astype(str)
qrels["docno"] = qrels["docno"].astype(str)

**Machine Learning**

In [19]:
# constants
RANK_CUTOFF = 100


In [23]:
bm25 = pt.BatchRetrieve(whole_index, wmodel="BM25")
tfidf = pt.BatchRetrieve(whole_index, wmodel="TF_IDF")
sdm = pt.rewrite.SDM()
ltr_feats_new1 = (bm25 % RANK_CUTOFF) >> pt.text.get_text(whole_index, ["recipe_name","keywords","description","ingredients"]) >> (
    pt.transformer.IdentityTransformer()
    ** # score of title (not originally indexed)
    (pt.text.scorer(body_attr="recipe_name", takes='docs', wmodel='BM25'))
    ** # score of title (not originally indexed)
    (pt.text.scorer(body_attr="keywords", takes='docs', wmodel='BM25'))
    ** 
    (pt.text.scorer(body_attr="description", takes='docs', wmodel='BM25'))
    ** 
    (pt.text.scorer(body_attr="ingredients", takes='docs', wmodel='BM25'))
)
# for reference, lets record the feature names here too
fnames=["f1","f2"]

In [29]:
SEED=42

from sklearn.model_selection import train_test_split

tr_va_topics, test_topics = train_test_split(topics, test_size=0.15, random_state=SEED)
train_topics, valid_topics =  train_test_split(tr_va_topics, test_size=0.17, random_state=SEED)

In [30]:
from sklearn.ensemble import RandomForestRegressor
pipeline = pt.FeaturesBatchRetrieve(whole_index, wmodel="BM25", features=["WMODEL:Tf"])
rf = RandomForestRegressor(n_estimators=400, verbose=1, random_state=42, n_jobs=2)
rf_pipe = ltr_feats_new1 >> pt.ltr.apply_learned_model(rf)
rf_pipe.fit(train_topics, qrels)


18:38:37.906 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
18:38:39.291 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
18:38:40.782 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
18:38:42.226 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Def

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.8s
[Parallel(n_jobs=2)]: Done 400 out of 400 | elapsed:    1.5s finished


In [31]:
pt.Experiment(
    [rf_pipe, bm25 % 50, tfidf % 50],
    test_topics,
    qrels,
    eval_metrics=["map", "ndcg", "ndcg_cut_10", "mrt"],
    names=["our model","bm25","tfidf"],
    baseline = 2
)

18:38:48.772 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
18:38:49.287 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
18:38:49.813 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
18:38:50.329 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Def

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.1s
[Parallel(n_jobs=2)]: Done 400 out of 400 | elapsed:    0.2s finished


,name,map,ndcg,ndcg_cut_10,mrt,map +,map -,map p-value,ndcg +,ndcg -,ndcg p-value,ndcg_cut_10 +,ndcg_cut_10 -,ndcg_cut_10 p-value
0,our model,0.445501,0.590654,0.663367,762.080302,4.0,1.0,0.131271,4.0,1.0,0.042508,0.0,5.0,0.103711
1,bm25,0.322835,0.459698,0.762654,150.398791,2.0,1.0,0.916070,3.0,0.0,0.343074,1.0,1.0,0.569462
2,tfidf,0.322784,0.456783,0.764173,139.052158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
